# I referred to [**the webpage**](https://qiita.com/halhorn/items/c91497522be27bde17ce) for the implementation.

In [7]:
import numpy as np

import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Layer, Embedding
from keras import backend as K
from keras.initializers import RandomNormal

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

In [2]:
class MultiheadAttention(Model):
    ## hidden_dim has to be multiples of head_num
    def __init__(self, hidden_dim=512, head_num=8, dropout_rate=0.1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_dim = hidden_dim
        self.head_num = head_num
        self.dropout_rate = dropout_rate
        
        self.q_dense_layer = Dense(hidden_dim, use_bias=False, name="q_dense_layer")
        self.k_dense_layer = Dense(hidden_dim, use_bias=False, name="k_dense_layer")
        self.v_dense_layer = Dense(hidden_dim, use_bias=False, name="v_dense_layer")
        self.output_dense_layer = Dense(hidden_dim, use_bias=False, name="output_dense_layer")
        self.attention_dropout_layer = Dropout(dropout_rate, name="attention_dropout_layer")
        
    def split_heads(self, x):
        batch_size, max_len, hidden_dim = tf.unstack(tf.shape(x))
        x = tf.reshape(x, [batch_size, max_len, self.head_num, self.hidden_dim // self.head_num])
        return tf.transpose(x, [0, 2, 1, 3])
    
    def combine_heads(self, heads):
        batch_size, _, max_len, _ = tf.unstack(tf.shape(heads))
        heads = tf.transpose(heads, [0, 2, 1, 3])
        return tf.reshape(heads, [batch_size, max_len, self.hidden_dim])
        
    def call(self, query, memory, attention_mask, train_flag):
        #two arguments of query and memory are already encoded as embedded vectors for all words
        q = self.q_dense_layer(query)
        k = self.k_dense_layer(memory)
        v = self.v_dense_layer(memory)
        
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        #for scaled dot-product
        depth_inside_each_head = self.hidden_dim // self.head_num
        q *= depth_inside_each_head ** -0.5
        
        score = tf.matmul(q, k, transpose_b=True)
        # query.dtype.min ≈ -∞
        score += tf.to_float(attention_mask) * query.dtype.min
        normalized_score = Activation("softmax")(score, name="attention_weight")
        normalized_score = self.attention_dropout_layer(normalized_score, training=train_flag)
        attention_weighted_output = tf.matmul(normalized_score, v)
        attention_weighted_output = self.combine_head(attention_weighted_output)
        return self.output_dense_layer(attention_weighted_output)

In [3]:
# SlefAttention class inherits MultiheadAttention class so that it can make query and memory come from the same source.
class SelfAttention(MultiheadAttention):
    
    def call(self, query, attention_mask, train_flag):
        return super().call(query, query, attention_mask, train_flag)

In [4]:
class PositionwiseFeedforwardNetwork(Model):
    
    def __init__(self, hidden_dim, dropout_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        
        self.first_dense_layer = Dense(hidden_dim*4, use_bias=True, activation="relu", name="first_dense_layer")
        self.second_dense_layer = Dense(hidden_dim, use_bias=True, activation="linear", name="second_dense_layer")
        self.dropout_layer = Dropout(dropout_rate, name="PFFN_dropout")
        
    def call(self, input, train_flag):
        # make the network more flexible to learn for the first dense layer(non-linear transformation is used),
        # and put the network back into the same hidden dim as original(linear transformation is used)
        x = self.first_dense_layer(input)
        x = self.dropout_layer(x, training=train_flag)
        return self.second_dense_layer(x)

In [5]:
class LayerNormalization(Layer):
    def build(self, input_shape):
        hidden_dim = input_shape[-1]
        self.scale = self.add_weight("layer_norm_scale", shape=[hidden_dim],
                                    initializer="ones")
        self.shift = self.add_weight("layer_norm_shift", shape=[hidden_dim],
                                    initializer="zeros")
        super().build(input_shape)
        
    def call(self, input, epsilon=1e-6):
        mean = K.mean(input, axis=[-1], keepdims=True)
        variance = K.var(input, axis=[-1], keepdims=True)
        normalized_input = (input - mean) / (K.sqrt(variance) + epsilon)
        return normalized_input * self.scale + self.shift

In [6]:
class PreLayerNormPostResidualConnectionWrapper(Model):
    def __init__(self, layer, dropout_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.layer = layer
        self.layer_norm = LayerNormalization()
        self.dropout_layer = Dropout(dropout_rate)
        
    def call(self, input, train_flag, *args, **kwargs):
        x = self.layer_norm(input)
        x = self.layer(x, training=train_flag, *args, **kwargs)
        output = self.dropout_layer(x, training=train_flag)
        return input + output

In [21]:
class AddPositionalEncoding(Layer):
    def call(self, input):
        data_type = input.dtype
        batch_size, max_len, emb_dim = tf.unstack(tf.shape(input))
        # i is from 0 to 255 when emb_dim is 512
        #so the doubled_i is from 0 to 510
        doubled_i = K.arange(emb_dim) // 2 * 2
        exponent = K.tile(K.expand_dims(doubled_i, 0), [max_len, 1])
        denominator_matrix = K.pow(10000.0, K.cast(exponent / emb_dim, data_type))
        
        # since cos(x) = sin(x + π/2), we convert the series of [sin, cos, sin, cos, ...]
        # into [sin, sin, sin, sin, ...]
        to_convert = K.cast(K.arange(emb_dim) % 2, data_type) * math.pi / 2
        convert_matrix = K.tile(tf.expand_dims(to_convert, 0), [max_len, 1])
        
        seq_pos = K.arange(max_len)
        numerator_matrix = K.cast(K.tile(K.expand_dims(seq_pos, 1), [1, emb_dim]), data_type)
        
        positinal_encoding = K.sin(numerator_matrix / denominator_matrix + convert_matrix)
        batched_positional_encoding = K.tile(K.expand_dims(positinal_encoding, 0), [batch_size, 1, 1])
        return input + batched_positional_encoding

In [ ]:
class TokenEmbedding(Layer):
    def __init__(self, vocab_size, emb_dim, data_type="float32", *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.data_type = data_type
        
    def build(self, input_shape):
        self.embedding_layer = Embedding(vocab_size,
                                   emb_dim,
                                   embedding_initializer=RandomNormal(mean=0.0, stddev=self.emb_dim**-0.5)
                                  )
        super().build(input_shape)
        
    def call(self, input):
        mask_for_pad = tf.to_float(tf.not_equal(input, PAD_ID))
        embedding = self.embedding_layer(input)
        pads_masked_embedding *= tf.expand_dims(mask, -1)
        return pads_masked_embedding * (self.emb_dim ** 0.5)